### 한빛아카데미 - 새로나온 책

- https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1&cate_cd=&- srt=&searchKey=&keyWord=(메인)
- https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B7192437175(서브)
- location='/academy/books/book_view.html?p_code=B7192437175'(알수있는 정보)
- 제목, 저자, 출간일, 쪽수, 가격, 역자

In [9]:
import requests
import pandas as pd # 결과는 DF로 볼 것이므로
from tqdm import tqdm
from bs4 import BeautifulSoup

1. 데이터 가져오기

In [10]:
base_url='https://www.hanbit.co.kr'
main_url = '/academy/books/new_book_list.html'
url = f'{base_url}{main_url}?page=1'
result = requests.get(url)
soup = BeautifulSoup(result.text)

2. 찾으려고 하는 데이터 찾기

In [11]:
lis = soup.select('li.sub_book_list')
len(lis)

20

- 3. 하나의 데이터로 원하는 정보 추출

In [12]:
li = lis[0]
href = li.select_one('.pop_quick_bg')['onclick']
href

"location='/academy/books/book_view.html?p_code=B7192437175'"

In [13]:
href = href[10:-1]
href

'/academy/books/book_view.html?p_code=B7192437175'

- sub page에서 정보 추출

In [14]:
sub_url = f'{base_url}{href}'
res = requests.get(sub_url)
book_soup = BeautifulSoup(res.text)

In [15]:
# 제목
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').text.strip()
title

'처음 배우는 매트랩'

In [16]:
# 저자
book_lis = info.select('.info_list > li')
author = ', '.join(book_lis[0].find('span').get_text().strip().split(' , '))
author

'방성완'

In [17]:
# 역자 

if len(book_lis) == 5:
    translator = '-'    # 없으면 - 표시
else:
    translator = ', '.join(book_lis[1].find('span').get_text().strip().split(' , '))
translator

'-'

In [18]:
# 출간일
date = book_lis[-4].find('span').get_text().strip()
date

'2023-04-10'

In [19]:
# 페이지
page = int(book_lis[-3].find('span').get_text().strip()[:-2])  # int까지하면 더 정교한 데이터 됨
page

416

In [20]:
# 가격
payments = book_soup.select_one('.payment_box.curr').find_all('p')
sales = payments[1].select_one('.pbr').find('strong').get_text().strip()
sales   # int로 바꾸려면: ,를 없애고 바꾸기

'30,000'

##### 4. 한 페이지에 있는 데이터

In [21]:
lines = []
for li in lis:
    href = li.select_one('.pop_quick_bg')['onclick'][10:-1]
    res = requests.get(base_url + href)
    book_soup = BeautifulSoup(res.text)
    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').text.strip()
    book_list = info.select('.info_list > li')
    author = ', '.join(book_list[0].find('span').get_text().strip().split(' , '))
    if len(book_lis) == 5:
        translator = '-'
    else:
        translator = ', '.join(book_lis[1].find('span').get_text().strip().split(' , '))
    date = book_lis[-4].find('span').get_text().strip()
    page = int(book_lis[-3].find('span').get_text().strip()[:-2])
    payments = book_soup.select_one('.payment_box.curr').find_all('p')
    sales = payments[1].select_one('.pbr').find('strong').get_text().strip()
    lines.append({
        '제목':title, '저자':author, '역자':translator,
        '출간일':date, '페이지수':page, '판매가':sales
    })

In [22]:
df1 = pd.DataFrame(lines)
df1

,제목,저자,역자,출간일,페이지수,판매가
0,처음 배우는 매트랩,방성완,-,2023-04-10,416,"30,000"
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,-,2023-04-10,416,"34,000"
2,정보교과교육론(3판),"최현종, 전용주",-,2023-04-10,416,"30,000"
3,"IT CookBook, 기초전자실험 with PSpice(3판)",홍순관,-,2023-04-10,416,"22,000"
4,경제수학 강의(3판),김성현,-,2023-04-10,416,"23,000"
5,글로벌 사회와 리더십(2판),안광현,-,2023-04-10,416,"28,000"
6,행동 재무론,고광수,-,2023-04-10,416,"32,000"
7,"IT CookBook, 알기 쉽게 해설한 파이썬 with 컴퓨팅 사고력","김충석, 김지연",-,2023-04-10,416,"24,000"
8,MATLAB으로 배우는 공학 수치해석(3판),방성완,-,2023-04-10,416,"28,000"
9,Cengel의 알기 쉬운 열역학(6판),"Yunus A. Çengel, John M. Cimbala, Afshin J. Gh...",-,2023-04-10,416,"34,000"
